In [ ]:
!git clone https://github.com/Paritosh97/SGNify.git
%cd SGNify

In [ ]:
!echo -e "\nDownloading SGNify..."
!wget --post-data "username=paritosh.sharmas@gmail.com&password=12345678" 'https://download.is.tue.mpg.de/download.php?domain=sgnify&resume=1&sfile=data.zip' -O 'data.zip' --no-check-certificate --continue
!unzip data.zip
!rm data.zip
!mkdir data/coco17
!mv data/vitpose-h-multi-coco.pth data/coco17/vitpose-huge.pth

In [ ]:
!GIT_LFS_SKIP_SMUDGE=1 git submodule update --init --recursive
!git submodule update --init --recursive

In [ ]:
%cd spectre
!pip3 install gdown
!echo -e "\nDownload pretrained SPECTRE model..."
!gdown --id 1vmWX6QmXGPnXTXWFgj67oHzOoOmxBh6B
!mkdir -p pretrained/
!mv spectre_model.tar pretrained/
import os

file_path = "data/template.mtl"

# Check if the file exists
if not os.path.isfile(file_path):
    # File does not exist, create it and write the content
    with open(file_path, "w") as file:
        file.write("newmtl FaceTexture\nmap_Kd mean_texture.jpg")
    print(f"File {file_path} created with the specified content.")
else:
    # File exists, overwrite it with the new content
    with open(file_path, "w") as file:
        file.write("newmtl FaceTexture\nmap_Kd mean_texture.jpg")
    print(f"File {file_path} already existed and has been overwritten with the specified content.")
# Create the target directory if it doesn't exist
!mkdir -p ./external/face_detection/ibug/face_detection/retina_face/weights/
# Download the files using wget
!wget "https://github.com/elliottzheng/face-detection/releases/download/0.0.1/mobilenet0.25_Final.pth" -O ./external/face_detection/ibug/face_detection/retina_face/weights/mobilenet0.25_Final.pth
!wget "https://github.com/elliottzheng/face-detection/releases/download/0.0.1/Resnet50_Final.pth" -O ./external/face_detection/ibug/face_detection/retina_face/weights/Resnet50_Final.pth
%cd ..
!cp -r data/FLAME2020 spectre/data/FLAME2020

In [ ]:
# Set the environment variable for PyOpenGL
os.environ['PYOPENGL_PLATFORM'] = 'egl'

# Install necessary packages
!apt-get update && apt-get install --no-install-suggests --no-install-recommends --yes \
    libglu1-mesa-dev freeglut3-dev

# Verify the installation of OpenGL related packages
!dpkg -l | grep -E 'libglu1-mesa-dev|freeglut3-dev'

!pip install -r requirements.txt

In [ ]:
# Top of main python script
!python sgnify.py --image_dir_path ./data/demo/frame_low --output_folder ./data/demo/output/frame_low   --output_name output_frame_low

In [ ]:
# Step 1: Update package lists to ensure the package information is up to date
!apt-get update

# Step 2: Install ffmpeg
!apt-get install -y ffmpeg

# Step 3: Create the necessary directories for storing data and video segments
!mkdir -p ./data/demo
!mkdir -p ./data/demo/frames

# Step 4: Download the video to the specified location
!wget -O ./data/demo/video.mp4 "https://nextcloud.lisn.upsaclay.fr/index.php/s/PZnG2sSFbRtrLEz/download"

# Step 5: Extract all frames from the video
!ffmpeg -i ./data/demo/video.mp4 -vf "select=not(mod(n\,1))" -vsync vfr ./data/demo/frames/frame%03d.png

# Step 6: Organize frames into folders, each containing video_frame_count / 10 frames
import os
import math

frame_dir = './data/demo/frames'
output_base_dir = './data/demo/segments'
frames = sorted([f for f in os.listdir(frame_dir) if f.endswith('.png')])

# Calculate the number of frames per folder
num_frames_per_folder = len(frames) // 10

# Create directories and move frames
for i in range(10):
    start_index = i * num_frames_per_folder
    end_index = start_index + num_frames_per_folder
    folder_name = f'{output_base_dir}/segment_{i:02d}'
    os.makedirs(folder_name, exist_ok=True)
    
    for frame in frames[start_index:end_index]:
        os.rename(f'{frame_dir}/{frame}', f'{folder_name}/{frame}')

# Handle any remaining frames (if len(frames) is not exactly divisible by 10)
remaining_frames = frames[10 * num_frames_per_folder:]
if remaining_frames:
    folder_name = f'{output_base_dir}/segment_10'
    os.makedirs(folder_name, exist_ok=True)
    for frame in remaining_frames:
        os.rename(f'{frame_dir}/{frame}', f'{folder_name}/{frame}')
!
!ls ./data/demo/frames

In [ ]:
import os

# Path to the base directory containing segment folders
base_segment_dir = './data/demo/segments'

# List all segment folders
segment_folders = sorted([f for f in os.listdir(base_segment_dir) if os.path.isdir(os.path.join(base_segment_dir, f))])

# Process each segment folder
for segment_folder in segment_folders:
    segment_folder_path = os.path.join(base_segment_dir, segment_folder)
    output_folder_path = os.path.join('./data/demo/output', f'1rjp_{segment_folder}')
    
    # Run the external Python script for each segment folder
    !python sgnify.py --image_dir_path {segment_folder_path} --output_folder {output_folder_path} --output_name output_frame_low

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
    
download_file('/kaggle/working/SGNify/data/demo/output/', 'out')

In [ ]:
!mkdir -p ./data/demo/output/video/
# Step 4: Download the video to the specified location
!wget "https://nextcloud.lisn.upsaclay.fr/index.php/s/MLwzywn5HG3XmQE/download"

In [ ]:
!mv ./download ./data/demo/output/video/tmp.zip
%cd ./data/demo/output/video/
!unzip ./tmp.zip
!mv tmp/ .tmp/
!rm tmp.zip
%cd ../../../../

In [30]:
!python sgnify.py --output_folder ./data/demo/output/video --resume

2024-07-23 11:20:05.962502: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 11:20:05.962619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 11:20:06.085400: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Resuming from after SPECTRE...
Running SGNify...
  1%|▏                                      | 3/490 [05:44<14:49:55, 109.64s/it]^C
Traceback (most recent call last):
  File "/kaggle/working/SGNify/smplifyx/main.py", line 261, in <module>
    main(**args)
  File "/kaggle/working/SGNify/smplifyx/main.py", line 232, in main
    fit_single_frame(img, keypoints[[0]],